# Spam grid croped by basin

This procedure will take 2 shapefile and perform the clip of them 

### Project configuration

for console comments...

In [ ]:
class bcolors:
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    ENDC = '\033[0m'
    UNDERLINE = '\033[4m'

We are going to import shapely fiona and rtree in order to win efficiency to manage the size of the data.

In [ ]:
import fiona
import rtree
from shapely.geometry import mapping, shape

In [5]:
%reload_ext version_information
%version_information shapely, fiona, rtree

Software versions
Python 3.5.2 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 5.2.2
OS Linux 4.9.8 moby x86_64 with glibc2.2.5
shapely 1.5.17.post1
fiona 1.7.0
rtree 0.8.2
Wed Feb 22 16:08:26 2017 UTC

The variables that store the data location

In [6]:
cropsLayer = '/home/jovyan/work/data/aqueduct/data_source/spam_cells/April_2013_IMPACT_Cell5M.shp'
waterRiskLayer = '/home/jovyan/work/data/aqueduct/data_source/basin_geo/aqueduct_global_dl_20150409.shp'
outlayer  = '/home/jovyan/work/data/aqueduct/dst/geom_test.shp'

In [8]:
with fiona.open(waterRiskLayer, 'r') as layer1:
    with fiona.open(cropsLayer, 'r') as layer2:
        # We copy schema and add the  new property for the new resulting shp
        crss=layer2.crs
        schema = layer2.schema.copy()
        schema['properties']['basinid'] = 'int:10'
        # We open a first empty shp to write new content from both others shp
        with fiona.open(outlayer, 'w', 'ESRI Shapefile', schema, crs=crss) as layer3:
            index = rtree.index.Index()
            for feat1 in layer1:
                fid = int(feat1['id'])
                geom1 = shape(feat1['geometry'])
                index.insert(fid, geom1.bounds)

            for feat2 in layer2:
                geom2 = shape(feat2['geometry'])
                for fid in list(index.intersection(geom2.bounds)):
                    if fid != int(feat2['id']):
                        feat1 = layer1[fid]
                        geom1 = shape(feat1['geometry'])
                        if geom1.intersects(geom2):
                            # We take attributes from waterRiskLayer
                            props = feat2['properties']
                            # Then append the uid attribute we want from the other shp
                            props['basinid'] = feat1['properties']['basinid']
                            # Add the content to the right schema in the new shp
                            layer3.write({
                                'properties': props,
                                'geometry': mapping(geom1.intersection(geom2))
                            });

OSError: no such file or directory: 'data/water_risk/aqueduct_projections_20150309.shp'